# Track notebooks & scripts

Call {meth}`~lamindb.core.Context.track` to track code along with its inputs and outputs.

```{note}

Tracking of notebooks & scripts is analogous to tracking data transformations through pipelines, functions & UI, see {doc}`docs:project-flow`.
```

In [ ]:
# !pip install 'lamindb[jupyter]'
!lamin init --storage ./test-track

## Initiate tracking

When you first call `ln.context.track()`, it raises an exception and creates a `stem_uid` & `version` to identify a notebook or script in your database.

When you call it a second time, `ln.context.track()` saves transform and run records in their registries. The {class}`~lamindb.Transform` registry allows you to find your notebooks, scripts and pipelines. The {class}`~lamindb.Run` registry allows you to find their runs.

In [ ]:
import lamindb as ln

ln.context.uid = "9priar0hoE5u0000"
ln.context.track()

LaminDB now automatically tracks all input and output data.

## Save run reports and source code

If you want to save the run report & source code for a notebook or script, call:

```python
ln.finish()
```

This is how a notebook with run report looks on the hub:

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/RGXj5wcAf7EAc6J8dJfH.png" width="900px">

## Query for a notebook or script

In the API, filter {class}`~lamindb.Transform` to obtain a transform record:

```python

transform = ln.Transform.filter(name="Track notebooks & scripts").one()
transform._source_code_artifact  # source code
transform.latest_run.report  # report of latest run
transform.runs  # all runs
```

On the hub, use the search or filter in the `Transform` view.

## Sync scripts with GitHub

To sync with your git commit, add the following line to your script:

```python
ln.settings.sync_git_repo = <YOUR-GIT-REPO-URL>
```

A tracked Python script could look like this:

```python
# my_script.py

import lamindb as ln

# initiate tracking
ln.settings.sync_git_repo = "https://github.com/..."
ln.settings.transform.stem_uid = "9priar0hoE5u"
ln.settings.transform.version = "1"
ln.context.track()

# load input artifacts
artifact = ln.Artifact.filter(...).one()
data = artifact.load()

output_data = ...

# save output artifacts
output_artifact = ln.Artifact(output_data, ...).save()

# save the script as transform._source_code_artifact
ln.finish()
```

You'll now see the GitHub emoji clickable on the hub:

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/IpV8Kiq4xUbgXhzlUYT7.png" width="900px">
<br>

In [ ]:
# clean up test instance
!lamin delete --force test-track